In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import os
from sklearn.preprocessing import LabelEncoder
import datetime
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizer

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data.read_parallel import read_parallel_local
from src.models.deeplegis import *
from src.models.data_loader import *
from src.models.configurationClasses import deepLegisConfig

from transformers import BertTokenizerFast
import pandarallel
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
config = deepLegisConfig("bert_128.json", project_root="../")
df, encoder = createDeepLegisDataFrame(config)

Original number of examples: 199646
Loading 199646 text files
Took 3.4365577975908916 min (206.1934735774994 sec)to open 199646 files with 20 processes.
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
df.to_pickle("/home/luke/ml_govhawk_prod_output/preprocessed_df_128.pkl")

In [40]:
tf.data.Dataset.from_tensor_slices((df.tokens.to_list()))
#tf.data.Dataset.from_tensor_slices([(1,3), (2,4)])

<TensorSliceDataset shapes: (128,), types: tf.int32>

In [39]:
as_list(df.tokens.values[0:3])

NameError: name 'as_list' is not defined

In [21]:
from pandarallel import pandarallel
pandarallel.initialize()
# Set CPU as available physical device
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer(df.text[0:1].to_list(), return_tensors="tf",  truncation=True, padding='max_length', max_length=128)

#tokens = tokenizer(df.text[0:2].to_list(), truncation=True, padding=True, max_length=128)
#print(len(tokens['input_ids'][0]))
#print(len(tokens['input_ids'][1]))

def tokenize(text):
    ids = tokenizer(text, truncation=True, padding=True, max_length=128)
    return tf.cast(ids['input_ids'], 'int32')
import time

tic = time.perf_counter()
df['tokenized'] = df.text.parallel_apply(tokenize)
toc = time.perf_counter()
print(f"Tokenized in {toc - tic:0.4f} seconds")


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Tokenized in 4.5029 seconds


In [6]:
df['tokenized'][0]

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  5900,  6372,  3180,  5219,  2000,  1024, 14814,  1037,
        2011,  1024,  4387,  6243,  2160,  3021,  2019,  2552,  2000,
       27950,  2930,  5900,  3642,  1997,  2000,  3166,  4697,  1996,
        5900,  4879,  1997, 27290,  2000, 27764,  1997,  4895, 25154,
        2098,  2512, 29278, 21156,  2098,  3200,  1999,  1996,  2168,
        5450,  2004, 15726,  3200,  1025,  1998,  2005,  3141,  5682,
        1012,  2022,  2009, 11955,  2011,  1996,  6372,  1997,  1996,
        2110,  1997,  5900,  1024,  2930,  1011,  2756,  1011, 20311,
        1010,  5900,  3642,  1997,  2003, 13266,  2004,  4076,  1024,
        3272,  2004,  4728,  3024,  1999,  1998,  2019,  3954,  1997,
        3200,  1010,  2060,  2084,  1037,  4758,  9415,  1010,  6315,
        3430,  2030, 11498, 27921, 12032,  2401,  1010,  2008,  2038,
        2042,  8243,  4618,  5371,  2019,  3437,  2306,  4228,  2420,
        2044,  1996,  6503,  1997,  2326,  1

In [20]:
df.tokenized[0]

{'input_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[  101,  5900,  6372,  3180,  5219,  2000,  1024, 14814,  1037,
         2011,  1024,  4387,  6243,  2160,  3021,  2019,  2552,  2000,
        27950,  2930,  5900,  3642,  1997,  2000,  3166,  4697,  1996,
         5900,  4879,  1997, 27290,  2000, 27764,  1997,  4895, 25154,
         2098,  2512, 29278, 21156,  2098,  3200,  1999,  1996,  2168,
         5450,  2004, 15726,  3200,  1025,  1998,  2005,  3141,  5682,
         1012,  2022,  2009, 11955,  2011,  1996,  6372,  1997,  1996,
         2110,  1997,  5900,  1024,  2930,  1011,  2756,  1011, 20311,
         1010,  5900,  3642,  1997,  2003, 13266,  2004,  4076,  1024,
         3272,  2004,  4728,  3024,  1999,  1998,  2019,  3954,  1997,
         3200,  1010,  2060,  2084,  1037,  4758,  9415,  1010,  6315,
         3430,  2030, 11498, 27921, 12032,  2401,  1010,  2008,  2038,
         2042,  8243,  4618,  5371,  2019,  3437,  2306,  4228,  2420,
         2044, 